In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [2]:
print('Lenght of dataset in characters: ', len(text))

Lenght of dataset in characters:  1115393


## All the unique characters that occur in this text

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## Tokenisation

Create a mapping for characters to integers

In [4]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

# Encoders take a string and output a list of integers
# Decoders take a list of integers and output a string

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi THERE!'))
print(decode(encode('hi THERE!')))

[46, 47, 1, 32, 20, 17, 30, 17, 2]
hi THERE!


Now lets encode the entire chunck of text we are having and save it to torch.tensor

In [5]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

## Spliting the data

Split the data in the validation and testing set

In [6]:
# We will keep 90 percent of proportion into testing and 10 percent to validate

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
# Usually we train the model of samples of data or block of data here we are calling them block_size

block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

A simple illustraion of working on the data with the specified block size

On what basis the predictions will be made

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is:{target}")

When input is tensor([18]) the target is:47
When input is tensor([18, 47]) the target is:56
When input is tensor([18, 47, 56]) the target is:57
When input is tensor([18, 47, 56, 57]) the target is:58
When input is tensor([18, 47, 56, 57, 58]) the target is:1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is:15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is:47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is:58


Now picking random blocks from the chunks starting from the random positions

In [9]:
torch.manual_seed(1020)
batch_size = 4  # How many independent sequences will we process in parallel?
block_size = 8  # What is the maximum context length for prediction?

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('Inputs: ')
print(xb.shape)
print(xb)
print('Output: ')
print(yb.shape)
print(yb)

print('_______')

for b in range(batch_size):  # Batch dimention
    for t in range(block_size):  # Time dimention
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} target is:{target}")

Inputs: 
torch.Size([4, 8])
tensor([[43,  1, 39, 52, 42,  1, 51, 63],
        [63, 53, 59, 56,  1, 46, 43, 39],
        [58, 46, 43,  1, 57, 47, 41, 49],
        [18, 47, 56, 57, 58,  1, 20, 59]])
Output: 
torch.Size([4, 8])
tensor([[ 1, 39, 52, 42,  1, 51, 63,  1],
        [53, 59, 56,  1, 46, 43, 39, 56],
        [46, 43,  1, 57, 47, 41, 49,  1],
        [47, 56, 57, 58,  1, 20, 59, 52]])
_______
When input is [43] target is:1
When input is [43, 1] target is:39
When input is [43, 1, 39] target is:52
When input is [43, 1, 39, 52] target is:42
When input is [43, 1, 39, 52, 42] target is:1
When input is [43, 1, 39, 52, 42, 1] target is:51
When input is [43, 1, 39, 52, 42, 1, 51] target is:63
When input is [43, 1, 39, 52, 42, 1, 51, 63] target is:1
When input is [63] target is:53
When input is [63, 53] target is:59
When input is [63, 53, 59] target is:56
When input is [63, 53, 59, 56] target is:1
When input is [63, 53, 59, 56, 1] target is:46
When input is [63, 53, 59, 56, 1, 46] target 

## Feeding to the model

Let us start with constructing a simple bigram model

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# Due to memory issue cannot use more than 20 max_new_tokens
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=10)[0].tolist()))

torch.Size([32, 65])
tensor(4.9503, grad_fn=<NllLossBackward0>)

Sr?qP-QWkt


## Optimization and model training

In [11]:
# Create a pytorch optimiser
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [12]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    
    # Evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.471766948699951


In [13]:
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


By y a!
Clit hn y, denjxhece w illd CHAh, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
Rie hicomyonthar's
PlinseKEd ith henouratucenonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h haybet--s n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scat t tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN the. sE:
I hisgothers je are! ABer wotouciullle'z,
Thitertho s?
NDan'spererfo cist ripl chys er orlese;
Yo jeh f h hecere ek? wf HEThot mowo soaf loit, ince his, t, f at. fal thetrimy bupof tor atha By!
JOutho f cimimave.
NENTIt cir sella p wie wede
Ro n apenor f'Y toverawitys an sh d w t e w!
CEOntiretoaveEd the we n ck. cung.
ORIsthies hacin benqurd bll, d a r w wistatsowor ath
Fivet bloll ail a-I theeancu,
LINCI'T:
Sarey t I Ane sze t
A hy thit,
n.
Fa

## Self attention block(basis of LLM)
It is used for processing these tokens

In [14]:
# Let us take a toy example first to understant the mathematical trick around self attention

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channel
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [15]:
# We want x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B,T,C)) # "bow" represents bag of characters
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [19]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei # Sum of each row in the matrix will be 1

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [25]:
# here torch will perform batch multiplication i.e will add a batch dimention to the wei and make it (B,T,T)
# the output maatrix will have dimentions (B,T,T) @ (B,T,C) ----> (B,T,C)
xbow2 = wei @ x

# If we check for similarity between xbow2 and xbow we will have full match
torch.allclose(xbow, xbow2)

False

In [26]:
# How can it be done using softmax

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [ ]:
# Ceating a single head self attention block

torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) # This will give (B,T,16) @ (B,16,T) ------> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = torch.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape